In [1]:
import csv
import os
import sys
import itertools
import numpy as np
import matplotlib.pyplot as plt
import requests
import pprint
import json
import glob
import pandas as pd
from requests.adapters import HTTPAdapter
import sys
!{sys.executable} -m pip install python-usda
from usda import UsdaClient
from requests.packages.urllib3.util.retry import Retry
import re
from collections import defaultdict

You are using pip version 18.1, however version 20.3.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
### USDA Client Key ###
# client = UsdaClient('ohrJBZWi8Ggdf00mMWbax5fTBFc4mRm0i8kFGu6D')
client = UsdaClient('5jy6h1p6kZf5UX6aBXdQr2frrtfvNZKNZrsyxCei')
USDA_API_KEY = '5jy6h1p6kZf5UX6aBXdQr2frrtfvNZKNZrsyxCei'

## Foursquare credential
CITY = 'Los Angeles'
CLIENT_ID = 'WB4STVV0BKOZGPDTSYPOKL1PCMN2ENFIE0UEJQF4PIMJPCGU' #Foursquare ID
CLIENT_SECRET = 'SNGR2XVXL4BTDRQBQJHLEYYE2ZXDCAQADNMOILCU34ZCGO4H' #Foursquare Secret
# CLIENT_ID = 'EKNXUN23A2RVQRND3DLNTNTBWTYPHP512RTLDVWFHHFPJBHX' 
# CLIENT_SECRET = 'F0TZIGY5BDBL3M4DT4SA3AEYON2RUXCVAQ3XCCFQHUAQ3RT4' 
# CLIENT_ID = 'E1WJ2PIB4UUZZLS2BPU5VZDHH5QIDDPIVCL3F0N1R2V1HH3Q' 
# CLIENT_SECRET = 'W3HFEUATX52S5325HZ501100YOWCZ02UZ5WSLVWRW0ZOGFEN' 
VERSION = '20180323'
# VERSION = '20181201'
venue_data_path = './venueData'
#CATEGORY_ID = '4bf58dd8d48988d14e941735' #Main Food category
# Food Category IDs from foursquares API
foodcategory_ids={"burgers":"4bf58dd8d48988d16c941735",
                "american":"4bf58dd8d48988d14e941735",
                "mexican":"4bf58dd8d48988d1c1941735",
                "fastfood":"4bf58dd8d48988d16e941735",
                "salad":"4bf58dd8d48988d1bd941735",
                "coffee_shops":"4bf58dd8d48988d1e0931735",
                "street_food":"53e0feef498e5aac066fd8a9",
                "seafood_restaurants":"4bf58dd8d48988d1ce941735",
                "piazza":"4bf58dd8d48988d1ca941735",
                "sandwich_place":"4bf58dd8d48988d1c5941735",
                "fried_chicken_joint":"4d4ae6fc7a7b7dea34424761",
                "chinese_restaurant":"4bf58dd8d48988d145941735",
                "korean_restaurant":"4bf58dd8d48988d113941735",
                "bakery":"4bf58dd8d48988d16a941735",
                "italian_restaurant":"4bf58dd8d48988d110941735",
                "taco_places":"4bf58dd8d48988d151941735",
                "sushi_restuarant":"4bf58dd8d48988d1d2941735",
                "brewery":"50327c8591d4c4b30a586d5d",
                "asian_restaurant":"4bf58dd8d48988d142941735",
                "steak_house":"4bf58dd8d48988d1cc941735",
                "bbq_joint":"4bf58dd8d48988d1df931735",
                }
venuenames_ids={}

In [3]:
def calculate_dailyvalue(nut1,nut, nutrient_name, nutrient,portion):
    nut[nutrient_name]=(nutrient.value/portion)*100
    if nutrient.name.lower() in nut1:
        nut1[nutrient_name]=nut1[nutrient_name]+((nutrient.value/portion)*100)
    else:
        nut1[nutrient_name]=((nutrient.value/portion)* 100)
                             

### 1. Read category venues and get a list of all restaurants under those categories using Foursquare API.
This step will run the foursquare API to open a list of venues. **DO NOT RUN this** if you don't need a new set of venue list becasuse there are limits to the API endpoint access.

In [4]:
print('Start scraping category venues...')
for i in foodcategory_ids:
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&categoryId={}&near={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET,
        VERSION, 
        foodcategory_ids[i], 
        CITY)
    params = dict(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    v='20180323',
    categoryId=foodcategory_ids[i],
    near=CITY
    # ll='40.7243,-74.0018',
    # query='coffee',
    # limit=1
    )
    results = requests.get(url=url, params=params).json()
    # results = requests.get(url).json()
    with open(os.path.join(venue_data_path, 'venuedata_'+i+'.json'), 'w+') as f:
        json.dump(results, f)


Start scraping category venues...


### 2. Get all venue id in order to get menus for all the venues found in the above.

In [ ]:
print('Get all venue id...')
all_venues_files = [f for f in glob.glob(os.path.join(venue_data_path, "venuedata_*.json"))]
all_venues={}

for i in all_venues_files:
    with open(i,'r') as results:
        data=json.load(results)
    venue=data['response']['venues']
    for i in venue:
        name=i['name']
        all_venues[name]=i['id']
# with open('LA.places.053120.csv', newline='') as csvfile:
#     la_csv = csv.reader(csvfile)
    # for row in la_csv:
        # print(row)
#     next(la_csv)
#     all_venues = [x[2] for x in la_csv]

### 3. Get menus for all the above venue ids

In [ ]:
# #3. Get menu's for all the above venue id's
# print('Getting menus from venue id...')
# menu_path = './menuData'
# for key, id in enumerate(all_venues):
# # for key, id in all_venues.items():
#     url = 'https://api.foursquare.com/v2/venues/{}/menu?client_id={}&client_secret={}&v={}'.format(
#                                         id, 
#                                         CLIENT_ID, 
#                                         CLIENT_SECRET, 
#                                         VERSION)
#     # always retry on 429: quota exceeded
#     retry_strategy = Retry(
#             total=3,
#             status_forcelist=[429, 500, 502, 503, 504],
#             method_whitelist=["HEAD", "GET", "OPTIONS"]
#         )
#     adapter = HTTPAdapter(max_retries=retry_strategy)
#     http = requests.Session()
#     http.mount("https://", adapter)
#     http.mount("http://", adapter)
#     response = requests.get(url)

#     results = response.json()
    
#     if response.status_code == 200:
#         print('Saving menu to csv....')
#         with open(os.path.join(menu_path, 'data_'+ ''.join(e for e in str(key) if e.isalnum()) +'.json'), 'w+') as f:
#             json.dump(results, f)

In [22]:
#3. Get menu's for all the above venue id's
print('Getting menus from venue id...')
menu_path = './'
venue_id_list = {'Taco Bell': '4c35731c3896e21e4249ed90', 'In-N-Out Burger': '49e3c0c1f964a520c7621fe3'}
for key, id in venue_id_list.items():
# for key, id in all_venues.items():
    url = 'https://api.foursquare.com/v2/venues/{}/menu?client_id={}&client_secret={}&v={}'.format(
                                        id, 
                                        CLIENT_ID, 
                                        CLIENT_SECRET, 
                                        VERSION)
    print(url)
    # always retry on 429: quota exceeded
    retry_strategy = Retry(
            total=3,
            status_forcelist=[429, 500, 502, 503, 504],
            method_whitelist=["HEAD", "GET", "OPTIONS"]
        )
    adapter = HTTPAdapter(max_retries=retry_strategy)
    http = requests.Session()
    http.mount("https://", adapter)
    http.mount("http://", adapter)
    response = requests.get(url)

    results = response.json()
    
    if response.status_code == 200:
        print('Saving menu to csv....')
        with open(os.path.join(menu_path, 'data_'+ ''.join(e for e in str(key) if e.isalnum()) +'.json'), 'w+') as f:
            json.dump(results, f)

Getting menus from venue id...
https://api.foursquare.com/v2/venues/4c35731c3896e21e4249ed90/menu?client_id=WB4STVV0BKOZGPDTSYPOKL1PCMN2ENFIE0UEJQF4PIMJPCGU&client_secret=SNGR2XVXL4BTDRQBQJHLEYYE2ZXDCAQADNMOILCU34ZCGO4H&v=20180323
Saving menu to csv....
https://api.foursquare.com/v2/venues/49e3c0c1f964a520c7621fe3/menu?client_id=WB4STVV0BKOZGPDTSYPOKL1PCMN2ENFIE0UEJQF4PIMJPCGU&client_secret=SNGR2XVXL4BTDRQBQJHLEYYE2ZXDCAQADNMOILCU34ZCGO4H&v=20180323
Saving menu to csv....


### 4. Scan each menu item to look for food item

In [19]:
menu_path = './menu_chain'# './Menus' # './menuData'
all_menu_files = [f for f in glob.glob(menu_path + "**/*.json", recursive=True)]

# menu_files = [pd.read_csv(f) for f in glob.glob(menu_path + "**/*.json", recursive=True)]
# menu_files = [json.load(pd.read_csv(f))['meta']['code'] == 200 for f in all_menu_files]

menu_files = []
for f in all_menu_files:
    with open(f,'r') as results:
        data=json.load(results)
        if data['meta']['code'] == 200 and data['response']['menu']['menus']['count'] > 0:
            menu_files.append(f)
print('Valid menu files: ', len(menu_files))

Valid menu files:  4


In [20]:
def usda_food_search(item):
    url = 'https://api.nal.usda.gov/fdc/v1/foods/search?api_key='+ USDA_API_KEY
    # USDA_API_KEY
    headers={'Content-type':'application/json', 'Accept':'application/json'}
    data = {
      "query": item,
      "pageSize": 25,
      "sortBy": "dataType.keyword",
      "sortOrder": "asc"
    }

    r = requests.post(url=url, json=data, headers=headers)
    if r.status_code :
        result = r.json()
        return result
    return None
        

### 5. Analyze the nutrional content for each using USDA nutrient database API

In [21]:
drink_corpus = ['wine', 'white', 'rosé', 'drink', 'drinks', 'red', 'cocktail', 'martinis', 'sparkling', 
                'cocktails', 'beverages', 'bptmargaritas', 'beer', 'sangria', 'tequila', 'anejo', 
               'mezcal', 'reposado', 'blanco / Plata', 'Aguas Frescas', 'bloodymary&mimosacart', 
                'seasonalspritz', 'aguafrescas', 'californiachardonnay', 'Italian White', 'French White', 
                'Cabernet', 'Italian Red', 'California Merlot', 'Pinot Noir', 'Bordeaux Red', 'White Wine', 
                'Red Wine', 'Rose / Sparkling', 'On Tap', 'Proudly Serving Forster & Burnett','Bubbles','Margaritas','Chardonnay','Vinos','Pinot Grigio','Sauvignon Blanc','Rose','Cócteles','Malbec','Cervezas','Bottled','Draft','Wines By the Glass','Margarita Pitchers','Tequila Shots',
                'Draught Beer', 'Old World Reds', 'Bottles & Cans', 'Eclectic Reds - New World', 'Cabernet Sauvignon', 'Bordeaux Blend', 'Prosecco', 'Champagne'
                'soda', 'coffee', 'Iced Tea', 'Aquas Frescas', 'Xv Coffee', 'merlot', 'coffee', 'Juice, Coffee and Tea'
               ]
drink_corpus = pd.read_csv('drinks_corpus.csv', delimiter=',', sep='\t', header=None)

def is_drink_item(item):
    item = str(item)
    clean_food_string = re.sub('\W+','', item.lower())
    for key in drink_corpus:
        clean_key = re.sub('\W+','', key.lower())
        if clean_food_string.find(clean_key) != -1 and clean_food_string == clean_key:
#             print('>>>>Filtered...: ', item)
            return True
    return False
        

In [22]:
def filter_drink_section(section):
    if not is_drink_item(menu_sub_section['name']):
        filtered_menu_sub_section.append(menu_sub_section)

In [23]:
food_dict = defaultdict(list)
menu_files_folder_name = 'nutrient_report_Nov_25' #### Change the name here for the nutrient report from USDA ####
def scan_food_item_from_main_section(menu_main_section):
    
    for menu_sub_section in menu_main_section:
        
        # filter drink section from the menu
        filtered_menu_sub_section = []
        if not is_drink_item(menu_sub_section['name']):
            filtered_menu_sub_section = menu_sub_section
            
        # add food items from menu's subsections
        if filtered_menu_sub_section != [] or filtered_menu_sub_section != None:
            if 'name' in filtered_menu_sub_section:
                print('menu section included: ', filtered_menu_sub_section['name'])
#                 print(filtered_menu_sub_section)
                if filtered_menu_sub_section['entries']['count'] > 1:  
                    for food in filtered_menu_sub_section['entries']['items']:
                        food_dict['name'].append(food['name'])
                        if 'description' in food:
                            food_dict['meta'].append(food['description'])

                else: 
                    if filtered_menu_sub_section['entries']['count'] > 0:
                        food_dict['name'].append(filtered_menu_sub_section['entries']['items'][0]['name'])
                        # add description to meta if any
                        if 'description' in filtered_menu_sub_section['entries']['items'][0]:
                            food_dict['meta'].append(filtered_menu_sub_section['entries']['items'][0]['description'])
            
    nut={}
    nut1={}
    count=0
    d={}
    main_nut=["energy","protein","total lipid (fat)","carbohydrate","fiber","sodium","cholesterol"]
    
    for food_name in food_dict['name']:
        #To get the nutrient decomposition for each.
        nut={"energy":0,"protein":0,"total lipid (fat)":0,"carbohydrate":0,"fiber":0,"sodium":0,"cholesterol":0}
        try:
            food_item =  re.sub('[^A-Za-z]+', ' ', food_name)
            report = usda_food_search(food_item)
            if report != None:
                with open(os.path.join(menu_files_folder_name + '/nutrient_'+ location_name + '_' + food_item.replace(" ", "_") + '.json'), 'w+') as file:
                    json.dump(report, file)

        except:
            continue

In [ ]:

total_nutrient_data={}

folder_name = "Menus" #### Change the name here for the menu data ####
menu_files = glob.glob(os.path.join(folder_name, "*.json"))
for f in menu_files:
    menu_main_section=defaultdict(list)
    total_nutrient_d={}
    location_name=f.split('/')[-1:][0].split('_')[1].split('.')[0]
    
    with open(f,'r') as results:
        print('scraping from ....', f)
        data=json.load(results)
        
        # add menu main sections to list
        if data['response']['menu']['menus']['count'] == 1:
            menu_main_section = [data['response']['menu']['menus']['items'][0]['entries']['items']]
        else:
            menu_main_section = [data['response']['menu']['menus']['items'][i]['entries']['items'] for i in range(len(data['response']['menu']['menus']['items']))]
    
    for main_section in menu_main_section:
        # filter drink section from the menu
        if not is_drink_item(main_section[0]['name']):
            scan_food_item_from_main_section(main_section)
            

scraping from .... Menus/data_Tocaya Organica.json
menu section included:  Bowls
menu section included:  Burritos
menu section included:  Tacos
menu section included:  Salads
menu section included:  Quesadillas
menu section included:  Sides / Soups
menu section included:  Fresh Drinks
menu section included:  Bottled Drinks
menu section included:  Dessert
menu section included:  Signature Catering Packages
menu section included:  Burrito Box
menu section included:  Taco Bars
menu section included:  Bowls
menu section included:  Salads
menu section included:  Sides & Extras
menu section included:  Sides
menu section included:  Extras
menu section included:  Standard Protein
menu section included:  Premium Protein
menu section included:  Taco Shells
menu section included:  Single Serve
menu section included:  Carafes
menu section included:  Iced Tea
menu section included:  Dessert
scraping from .... Menus/data_The Misfit Restaurant + Bar.json
menu section included:  Raw Bar
menu section i

### 6. Process the results from USDA Food Database and print CSV

In [11]:
import json
# menu_files_folder_name = 'nutrient_report_Oct_7_new'
# nutrient_reports = glob.glob(os.path.join(menu_files_folder_name, "*.json"))

# Load all nutrient report
nutrient_reports = []
nutrient_report_names = ['nutrient_report_Nov_25', 'nutrient_reports_4_chain'] #### Change the name here for the nutrient report from USDA ####
for files in nutrient_report_names:
    nutrient_reports.extend(glob.glob(os.path.join(files, "*.json")))

# Setup CSV
# menu_analysis="./nutrient_analysis_"+ menu_files_folder_name +".csv"
menu_analysis="./nutrient_analysis_Nov_25.csv" #### FINAL CSV filename here ####
csvfile=open(menu_analysis,'w')
menu_analysis_f = csv.writer(csvfile)
menu_analysis_f.writerow(["Restaurant Name","Original Menu Item", "Matched food Item", "Match Score","Energy (KCAL)","Protein (G)","Total lipid(fat) (G)","Carbohydrate (G)","Fiber (G)","Sodium (MG)","Cholesterol (MG)"])

# Average all nutrient content 
food_score_threshold = 25

for report in nutrient_reports:
    energy, protein, total_lipid, carbohydrate, fiber, sodium, cholesterol = [],[],[],[],[],[],[] 
    with open(str(report)) as input_file:
        filename = report.split('/')[-1].split('_')
        location_name = filename[1]
        food = report.split('/')[-1].split('.json')[0].split(location_name)[-1].replace('_', ' ').lstrip()
        data = json.load(input_file)
        if 'foods' in data:
            food_benchmark = data["foods"]
            match_food_items = []
            match_food_score = 0
            pos_correlated_food_search = []
            for food_item in food_benchmark:
                pos_correlated_food_search.append(False)
                match_food_items.append(food_item['description'])
                match_food_score += food_item['score']
                food_nutrients = food_item['foodNutrients']

                # Calculate food value if matched food item is positively correlated to original food item
                if food_nutrients and match_food_score > food_score_threshold:
                    pos_correlated_food_search.append(True)
                    for food_nutrient in food_nutrients:
                        if food_nutrient['nutrientId'] == 1003:
                            protein.append(food_nutrient['value'])
                        if food_nutrient['nutrientId'] == 1004:
                            total_lipid.append(food_nutrient['value'])
                        if food_nutrient['nutrientId'] == 1008:
                            energy.append(food_nutrient['value'])
                        if food_nutrient['nutrientId'] == 1005:
                            carbohydrate.append(food_nutrient['value'])
                        if food_nutrient['nutrientId'] == 1079:
                            fiber.append(food_nutrient['value'])
                        if food_nutrient['nutrientId'] == 1093:
                            sodium.append(food_nutrient['value'])
                        if food_nutrient['nutrientId'] == 1253:
                            cholesterol.append(food_nutrient['value'])
    
    # Calculate the average nutrients
    if np.array(pos_correlated_food_search).any():
        average_protein = sum(protein) / len(protein) if len(protein) > 0 else 0
        average_total_lipid = sum(total_lipid) / len(total_lipid) if len(total_lipid) > 0 else 0
        average_energy = sum(energy) / len(energy) if len(energy) > 0 else 0
        average_carbohydrate = sum(carbohydrate) / len(carbohydrate) if len(carbohydrate) > 0 else 0
        average_fiber = sum(fiber) / len(fiber) if len(fiber) > 0 else 0
        average_sodium = sum(sodium) / len(sodium) if len(sodium) > 0 else 0
        average_cholesterol = sum(cholesterol) / len(cholesterol) if len(cholesterol) > 0 else 0
        
        average_match_food_score = match_food_score / len(match_food_items) * 1.0 if match_food_score != 0 else 0

        menu_analysis_f.writerow([location_name,food, match_food_items, average_match_food_score, average_energy,average_protein,average_total_lipid,average_carbohydrate, average_fiber,average_sodium,average_cholesterol])


### 7. Display top 50 menu nutrient analysis result

In [12]:
menu_analytic_report_name = "./nutrient_analysis_Nov_25.csv" #### FINAL CSV filename here ####
nutrient_pd = pd.read_csv(menu_analytic_report_name) 
print('This table contains: {} rows x {} cols.'.format(nutrient_pd.shape[0], nutrient_pd.shape[1]))
nutrient_pd.head(50)

This table contains: 3051 rows x 11 cols.


,Restaurant Name,Original Menu Item,Matched food Item,Match Score,Energy (KCAL),Protein (G),Total lipid(fat) (G),Carbohydrate (G),Fiber (G),Sodium (MG),Cholesterol (MG)
0,The Rose Venice,Baja Taco,"['BAJA TACO PREMIUM VIRGINIA ARTISAN PEANUTS, ...",5.911944,202.823529,7.595556,11.626667,15.683333,2.583333,1454.944444,32.307692
1,Bob,Fish Tacos,"['FISH TACOS', 'FISH TACOS', 'FISH TACO SAUCE,...",138.583557,186.320000,6.794400,9.279200,16.902917,3.800000,5193.160000,20.882353
2,The Hat,Al Pastor,"['AL PASTOR TACO SEASONING, AL PASTOR', 'AL PA...",165.468484,124.083333,6.254583,4.808750,22.214167,3.150000,1613.833333,4297.071429
3,The Apple Pan,Pork Chops,"['PORK CHOPS', 'PORK CHOPS', 'PORK CHOPS', 'BO...",150.941473,157.920000,18.192000,7.682800,6.044000,0.860000,1203.880000,52.458333
4,Toca Madera,Breakfast Sandwich,"['BREAKFAST SANDWICH', 'BREAKFAST SANDWICHES',...",61.269250,285.120000,11.347200,14.572800,28.084000,2.952000,532.880000,58.640000
5,Blue Plate Taco,French Toast,"['BREAKFAST SANDWICH', 'BREAKFAST SANDWICHES',...",61.269250,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,The Apple Pan,Carne Asada Eggs,"['CARNE ASADA MARINADE, CARNE ASADA', 'CARNE A...",437.891363,137.208333,8.388333,4.760952,18.458696,2.207143,4530.625000,20.625000
7,Casa Vega,Plantain Chips,"['PLANTAIN CHIPS', 'PLANTAIN CHIPS', 'PLANTAIN...",1093.901472,516.000000,3.368000,26.340000,65.948000,4.476000,239.760000,0.000000
8,THE Blvd,Housemade Vanilla Lime Soda,"['PLANTAIN CHIPS', 'PLANTAIN CHIPS', 'PLANTAIN...",1093.901472,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Casa Vega,Pan Seared Salmon,"['PAN SEARED TERIYAKI SALMON, PAN SEARED TERIY...",39.375816,169.960000,15.977600,5.178400,14.929600,0.500000,710.720000,40.833333
